<a href="https://colab.research.google.com/github/nrrajuu/Natural-Language-Processing/blob/main/Match_Winning_Analysis%20with%20Machine%20Learning%20with%20text%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
import re
import string
import nltk

import numpy as np
import pandas as pd

#nltk.download()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize




#links
#https://sparkbyexamples.com/pandas/pandas-get-cell-value-from-dataframe/ - show a cell in a data from
#https://www.youtube.com/watch?v=isuRxhLQSXU - how to clean up text
#https://dylancastillo.co/text-classification-using-python-and-scikit-learn/#prerequisites

In [ ]:
# importing Sales Data and Placing it into a Data Frame

#categories = [
 #   "solution",
  #  "pricing",
  #  "relationships and incumbency",
  #  "value based selling",
#]

sales_data = pd.read_csv('FinalData.csv',encoding='ISO-8859-1')

df = pd.DataFrame(
  dict(
        value=sales_data["Value"],
        outcome=sales_data["Outcome"],
        length=sales_data["Length"],
        competitors=sales_data["Competitors"],
        incumbent=sales_data["Incumbency"],
        sales_notes=sales_data["Notes"],
        classification=sales_data["Classification"]
  )
)

#df = df.dropna(inplace=True)


In [ ]:
#create a new lenght column in the data frame to store the lenght caterogy that the sales deal is assigned to

df["new_length"] = ""

#update lengh value with string based on how long sales cycle took - six month increments up to 5 years
df.loc[df['length'] <= 180, 'new_length'] = str("6monthsless") #6 months or less
df.loc[(df['length'] >= 181) & (df['length'] <= 360) , 'new_length'] = str("6to12months") #12 months or less
df.loc[(df['length'] >= 361) & (df['length'] <= 540) , 'new_length'] = str("12to18months") #18 months or less
df.loc[(df['length'] >= 541) & (df['length'] <= 720) , 'new_length'] = str("18to24months") #24 months or less
df.loc[(df['length'] >= 721) & (df['length'] <= 900) , 'new_length'] = str("24to30months") #30 months or less
df.loc[(df['length'] >= 901) & (df['length'] <= 1080) , 'new_length'] = str("30to36months") #36 months or less
df.loc[(df['length'] >= 1081) & (df['length'] <= 1260) , 'new_length'] = str("36to42months") #42 months or less
df.loc[(df['length'] >= 1261) & (df['length'] <= 1440) , 'new_length'] = str("42to48months") #48 months or less
df.loc[(df['length'] >= 1441) & (df['length'] <= 1620) , 'new_length'] = str("48to54months") #54 months or less
df.loc[(df['length'] >= 1621) & (df['length'] <= 1800) , 'new_length'] = str("54to60months") #60 months or less
df.loc[df['length'] >= 1801, 'new_length'] = str("over5years") #Over 5 years


df

,value,outcome,length,competitors,incumbent,sales_notes,classification,new_length
0,151000,Won,121,CompetitorA,UnknownIncumbent,We lost the wireless in this account some year...,Solution,6monthsless
1,883000,Lost,654,CompetitorA,UnknownIncumbent,CompetitorA closed deal at 100000 less,Pricing,18to24months
2,30000,Won,245,CompetitorA,CompanyIncumbent,CompetitorA CompAFeature001 fails to provide s...,Solution,6to12months
3,625000,Won,678,CompetitorA,CompetitorIncumbent,Company had a better solution for wireless tha...,Solution,18to24months
4,500000,Lost,377,CompetitorA and CompetitorB,UnknownIncumbent,CompetitorB offer has been less than the half ...,Pricing,12to18months
...,...,...,...,...,...,...,...,...
395,106000,Won,15,CompetitorI,UnKnownIncumbent,Team led with best discount to secure the deal,Pricing,6monthsless
396,18000,Won,57,CompetitorI,CompetitorIncumbent,Customer has CompetitorI products in their cu...,Solution,6monthsless
397,1000,Lost,101,CompetitorI,CompetitorIncumbent,Customer has already CompetitorI infrastructur...,Customer Relationships and Incumbency,6monthsless
398,55000,Lost,108,CompetitorI,UnKnownIncumbent,RFP points against CompetitorI Powerone,Value Based Selling,6monthsless


In [ ]:
#create a new value column in the data frame to store the value caterogy that the sales deal is assigned to

df["new_value"] = ""

#update dollar value with string based on how much the deal was worth - million dollar increments
df.loc[df['value'] <= 1000000, 'new_value'] = str("1millionless") #1 million or less
df.loc[(df['value'] >= 1000001) & (df['value'] <= 2000000) , 'new_value'] = str("1to2million") #2 million or less
df.loc[(df['value'] >= 2000001) & (df['value'] <= 3000000) , 'new_value'] = str("2to3million") #3 million or less
df.loc[(df['value'] >= 3000001) & (df['value'] <= 4000000) , 'new_value'] = str("3to4million") #4 million or less
df.loc[(df['value'] >= 4000001) & (df['value'] <= 5000000) , 'new_value'] = str("4to5million") #5 million or less
df.loc[(df['value'] >= 5000001) & (df['value'] <= 6000000) , 'new_value'] = str("5to6million") #6 million or less
df.loc[(df['value'] >= 6000001) & (df['value'] <= 7000000) , 'new_value'] = str("6to7million") #7 million or less
df.loc[(df['value'] >= 7000001) & (df['value'] <= 8000000) , 'new_value'] = str("7to8million") #8 million or less
df.loc[(df['value'] >= 8000001) & (df['value'] <= 9000000) , 'new_value'] = str("8to9million") #9 million or less
df.loc[(df['value'] >= 9000001) & (df['value'] <= 10000000) , 'new_value'] = str("9to10million") #10 million or less
df.loc[(df['value'] >= 10000001) & (df['value'] <= 11000000) , 'new_value'] = str("10to11million") #11 million or less
df.loc[(df['value'] >= 11000001) & (df['value'] <= 12000000) , 'new_value'] = str("11to12million") #12 million or less
df.loc[(df['value'] >= 12000001) & (df['value'] <= 13000000) , 'new_value'] = str("12to13million") #13 million or less
df.loc[df['value'] >= 13000001, 'new_value'] = str("over13million") #Over 5 years


df

,value,outcome,length,competitors,incumbent,sales_notes,classification,new_length,new_value
0,151000,Won,121,CompetitorA,UnknownIncumbent,We lost the wireless in this account some year...,Solution,6monthsless,1millionless
1,883000,Lost,654,CompetitorA,UnknownIncumbent,CompetitorA closed deal at 100000 less,Pricing,18to24months,1millionless
2,30000,Won,245,CompetitorA,CompanyIncumbent,CompetitorA CompAFeature001 fails to provide s...,Solution,6to12months,1millionless
3,625000,Won,678,CompetitorA,CompetitorIncumbent,Company had a better solution for wireless tha...,Solution,18to24months,1millionless
4,500000,Lost,377,CompetitorA and CompetitorB,UnknownIncumbent,CompetitorB offer has been less than the half ...,Pricing,12to18months,1millionless
...,...,...,...,...,...,...,...,...,...
395,106000,Won,15,CompetitorI,UnKnownIncumbent,Team led with best discount to secure the deal,Pricing,6monthsless,1millionless
396,18000,Won,57,CompetitorI,CompetitorIncumbent,Customer has CompetitorI products in their cu...,Solution,6monthsless,1millionless
397,1000,Lost,101,CompetitorI,CompetitorIncumbent,Customer has already CompetitorI infrastructur...,Customer Relationships and Incumbency,6monthsless,1millionless
398,55000,Lost,108,CompetitorI,UnKnownIncumbent,RFP points against CompetitorI Powerone,Value Based Selling,6monthsless,1millionless


In [ ]:
#combine all input features that will be feed into the algorithm into one column

df["inputs"] =   df['outcome'] + ' ' + df['sales_notes'] + ' ' + df['competitors']

#df['new_value']
#df['new_length']
#df['incumbent']
#df['competitors']
df

,value,outcome,length,competitors,incumbent,sales_notes,classification,new_length,new_value,inputs
0,151000,Won,121,CompetitorA,UnknownIncumbent,We lost the wireless in this account some year...,Solution,6monthsless,1millionless,Won We lost the wireless in this account some ...
1,883000,Lost,654,CompetitorA,UnknownIncumbent,CompetitorA closed deal at 100000 less,Pricing,18to24months,1millionless,Lost CompetitorA closed deal at 100000 less Co...
2,30000,Won,245,CompetitorA,CompanyIncumbent,CompetitorA CompAFeature001 fails to provide s...,Solution,6to12months,1millionless,Won CompetitorA CompAFeature001 fails to provi...
3,625000,Won,678,CompetitorA,CompetitorIncumbent,Company had a better solution for wireless tha...,Solution,18to24months,1millionless,Won Company had a better solution for wireless...
4,500000,Lost,377,CompetitorA and CompetitorB,UnknownIncumbent,CompetitorB offer has been less than the half ...,Pricing,12to18months,1millionless,Lost CompetitorB offer has been less than the ...
...,...,...,...,...,...,...,...,...,...,...
395,106000,Won,15,CompetitorI,UnKnownIncumbent,Team led with best discount to secure the deal,Pricing,6monthsless,1millionless,Won Team led with best discount to secure the ...
396,18000,Won,57,CompetitorI,CompetitorIncumbent,Customer has CompetitorI products in their cu...,Solution,6monthsless,1millionless,Won Customer has CompetitorI products in thei...
397,1000,Lost,101,CompetitorI,CompetitorIncumbent,Customer has already CompetitorI infrastructur...,Customer Relationships and Incumbency,6monthsless,1millionless,Lost Customer has already CompetitorI infrastr...
398,55000,Lost,108,CompetitorI,UnKnownIncumbent,RFP points against CompetitorI Powerone,Value Based Selling,6monthsless,1millionless,Lost RFP points against CompetitorI Powerone C...


In [ ]:
#Define a function to clean and process the inputs column

ps = PorterStemmer()


def process_inputs (inputs):
    inputs = str(inputs).lower()
    inputs = re.sub(
        f"[{re.escape(string.punctuation)}]", " ", inputs
    )
    inputs = " ".join(inputs.split())
    return inputs

df["inputs"] = df.inputs.map(process_inputs)
df["inputs"] = [ps.stem(word) for word in (df.inputs)]

#print(df.iloc[2,9])

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.30, stratify=df.classification)

In [ ]:
vectorizer = CountVectorizer(
     ngram_range=(1, 3),
     stop_words="english",
      )

X_train = vectorizer.fit_transform(df_train.inputs)
X_test = vectorizer.transform(df_test.inputs)

y_train = df_train.classification
y_test = df_test.classification

In [ ]:
#Now we train our second model which is a logistic regression model
#Number of times the model will go through training and comparing the data to improve overall accuracy

model = LogisticRegression(max_iter = 2000, solver = 'sag')

model.fit(X_train,y_train)

classify_lr = model.predict(X_test)

#accuracy_score(y_test,classify)
print(classification_report(y_test, classify_lr))

                                       precision    recall  f1-score   support

Customer Relationships and Incumbency       0.38      0.17      0.23        18
                              Pricing       0.59      0.78      0.67        46
                             Solution       0.58      0.66      0.62        44
                  Value Based Selling       1.00      0.08      0.15        12

                             accuracy                           0.57       120
                            macro avg       0.64      0.42      0.42       120
                         weighted avg       0.60      0.57      0.53       120



In [ ]:
classifier = RandomForestClassifier(n_estimators=2000, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

                                       precision    recall  f1-score   support

Customer Relationships and Incumbency       0.00      0.00      0.00        18
                              Pricing       0.60      0.80      0.69        46
                             Solution       0.53      0.70      0.61        44
                  Value Based Selling       0.00      0.00      0.00        12

                             accuracy                           0.57       120
                            macro avg       0.28      0.38      0.32       120
                         weighted avg       0.42      0.57      0.49       120



/Users/aleetabell/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aleetabell/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aleetabell/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
